In [ ]:
## LSTM Network
import time
import pandas as pd
#### Parameters

# For Training Data
start = '2015-01-01'  ## FIX THIS
end = '2020-10-13'  ## LAST TRAINING DATE + 1
stock = 'APOLLOTYRE.NS'    ## Choose one of ['BALKRISIND.NS', 'APOLLOTYRE.NS', 'CEATLTD.NS', 'JKTYRE.NS', 'TVSSRICHAK.NS', 'PTL.NS', 'ELGIRUBCO.NS']
# feature = 'High'  ## Select one of ['Open'; 'High' ; 'Low' ; 'Close'] in STRINGS

## For Creating dataset
time_step = 100     ## No. of days to be 'memorized' by model

## For forecasting
forecast_days = 11   ## No. of days to be forecasted after training end date

## For Verification with Actual Data
start_ver = '2020-10-7'  ## PREDICTION START DATE = (LAST TRAINING DATE - 2)
end_ver = '2020-10-10'  ## PREDICTION END DATE(TODAY'S DATE)

## Initialize Predictions DataFrame
Predictions_All = pd.DataFrame()

## START
main_tic = time.perf_counter()
# Install yfinance package. 
!pip install yfinance 
 
# Import yfinance 
import yfinance as yf   
 
ticks = ['BALKRISIND.NS', 'APOLLOTYRE.NS', 'CEATLTD.NS', 'JKTYRE.NS', 'TVSSRICHAK.NS', 'PTL.NS', 'ELGIRUBCO.NS']
labels = ['Balkrishna Industries', 'Apollo', 'CEAT', 'JKTyre', 'TVS', 'PTL', 'Elgi Rubber']
colours = ['red', 'green', 'blue', 'cyan', 'magenta', 'yellow', 'black']


# # Plot the close prices 
# import matplotlib.pyplot as plt 
# plt.figure(figsize=(20,20)) 
# plt.subplot(1,1,1)

# for i in range(len(ticks)):
#   data = yf.download(ticks[i],'2015-01-01','2020-10-05') 
#   plt.plot(data.Close, color = colours[i], label = labels[i], linewidth=4.0) 
# plt.legend(loc=2, prop={'size': 20})
# plt.xticks(size=20)
# plt.yticks(size=20)
# plt.title("Stock Prices for companies in the Tyre Sector in NSE from 1 January 2015", size=20)
# plt.show()

import numpy as np
import pandas as pd

data = yf.download(stock,start,end) 
data.reset_index(inplace=True)
data2 = data[['Date', 'Open', 'High', 'Low', 'Close']]
print(data2.tail())

for feature in ['Open', 'High', 'Low', 'Close']:
    print("#################################### " + feature + " ###########################################")
    tic = time.perf_counter()
    df1 = data2[feature]

    # df1

    ### LSTM are sensitive to the scale of the data. so we apply MinMax scaler

    from sklearn.preprocessing import MinMaxScaler
    scaler=MinMaxScaler(feature_range=(0,1))
    df1=scaler.fit_transform(np.array(df1).reshape(-1,1))

    # print(df1.shape,df1)

    ##splitting dataset into train and test split
    training_size=int(len(df1)*0.65)
    test_size=len(df1)-training_size
    train_data,test_data=df1[0:training_size,:],df1[training_size:len(df1),:1]

    # training_size,test_size

    import numpy
    # convert an array of values into a dataset matrix

    def create_dataset(dataset, time_step=1):
      dataX, dataY = [], []
      for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
      return numpy.array(dataX), numpy.array(dataY)

    # reshape into X=t,t+1,t+2,t+3 and Y=t+4
    time_step = time_step

    X_train, y_train = create_dataset(train_data, time_step)
    X_test, ytest = create_dataset(test_data, time_step)

    # print(X_train.shape), print(y_train.shape)

    # print(X_test.shape), print(ytest.shape)

    ## reshape input to be [samples, time steps, features] which is required for LSTM
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    ### Create the Stacked LSTM model
    import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense
    from tensorflow.keras.layers import LSTM
    # print(tf.__version__)

    from keras.callbacks import History 
    history = History()

    model=Sequential()
    model.add(LSTM(50,return_sequences=True,input_shape=(X_train.shape[1],1)))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(25))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',optimizer='adam')

    #model.summary()

    hist = model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=64,verbose=1, callbacks=[history])

    ### Lets Do the prediction and check performance metrics
    train_predict=model.predict(X_train)
    test_predict=model.predict(X_test)

    ##Transformback to original form
    train_predict=scaler.inverse_transform(train_predict)
    test_predict=scaler.inverse_transform(test_predict)

    ### Calculate RMSE performance metrics
    import math
    from sklearn.metrics import mean_squared_error
    print("Train MSE: ", math.sqrt(mean_squared_error(y_train,train_predict)))    #Should be 217.44307908665783

    ### Test Data RMSE    #Should be 162.29651852385174
    print("Test MSE: ", math.sqrt(mean_squared_error(ytest,test_predict)))

    # ### Plotting 
    # # shift train predictions for plotting
    # look_back=100
    # trainPredictPlot = numpy.empty_like(df1)
    # trainPredictPlot[:, :] = np.nan
    # trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
    # # shift test predictions for plotting
    # testPredictPlot = numpy.empty_like(df1)
    # testPredictPlot[:, :] = numpy.nan
    # testPredictPlot[len(train_predict)+(look_back*2)+1:len(df1)-1, :] = test_predict
    # # plot baseline and predictions
    # plt.plot(scaler.inverse_transform(df1), label='Original Data')
    # plt.plot(trainPredictPlot, label='Trained on')
    # plt.plot(testPredictPlot, label='Prediction')
    # plt.show()

    # len(test_data)

    x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
    # x_input.shape

    temp_input=list(x_input)
    temp_input=temp_input[0].tolist()

    # demonstrate prediction for next 10 days
    from numpy import array

    forecast_days = forecast_days
    n_steps = time_step

    lst_output=[]
    print("Normalized Predictions:\n")

    i=0
    while(i<forecast_days):
        
        if(len(temp_input)>time_step):
            #print(temp_input)
            x_input=np.array(temp_input[1:])
            #print("{} day input {}".format(i,x_input))
            x_input=x_input.reshape(1,-1)
            x_input = x_input.reshape((1, n_steps, 1))
            #print(x_input)
            yhat = model.predict(x_input, verbose=0)
            # print("{} day output: {}".format(i+1,yhat[0][0]))     ## remove this only
            temp_input.extend(yhat[0].tolist())
            temp_input=temp_input[1:]
            #print(temp_input)
            lst_output.extend(yhat.tolist())
            i=i+1
        else:
            x_input = x_input.reshape((1, n_steps,1))
            yhat = model.predict(x_input, verbose=0)
            # print("1 day output: {}".format(yhat[0][0]))    ## remove this only
            temp_input.extend(yhat[0].tolist())
            #print(len(temp_input))
            lst_output.extend(yhat.tolist())
            i=i+1
        
    #print(lst_output)

    model.save("saved_model.h5")

    day_new=np.arange(1,time_step+1)
    day_pred=np.arange(time_step,time_step + forecast_days)

    import matplotlib.pyplot as plt

    # len(df1)

    Predictions = scaler.inverse_transform(lst_output)
    # plt.plot(day_new,scaler.inverse_transform(df1[len(df1)-time_step:]), label='Days for which model was trained')
    # plt.plot(day_pred,Predictions, label='Predictions based on that data') 
    # plt.legend()
    # plt.show()

    df3=df1.tolist()
    df3.extend(lst_output)
    df3=scaler.inverse_transform(df3).tolist()
    # plt.plot(df3)
    # plt.axvline(x=len(df3)-len(Predictions), ls=':', c='black')
    # plt.title("Entire graph along with predictions")

    """## Verify Actual Data"""

    Predictions = Predictions.tolist()

    # start_ver = start_ver ## PREDICTION START DATE = LAST TRAINING DATE - 2
    # end_ver = end_ver  ## PREDICTION END DATE(TODAY'S DATE)

    # data3 = yf.download(stock,start_ver,end_ver) 
    # data3.reset_index(inplace=True)
    # data3 = data3[['Date', 'Open', 'High', 'Low', 'Close']]
    # print(data3) # Actual data
    # print(Predictions)

    # plt.figure(figsize=(12,6))
    # plt.plot(day_new,scaler.inverse_transform(df1[len(df1)-time_step:]), label='Days for which model was trained')
    # plt.plot(day_pred,Predictions, label='Predictions based on that data') 
    # plt.plot(day_pred[:len(data3[feature])],data3[[feature]], c='magenta', label='Actual Values')  ## MAKE SURE LENGTH OF DATA3 IS SAME AS LENGTH OF PREDICTIONS
    # plt.legend()
    # plt.title("Predictions and Actual Data")
    # plt.show()

    """### Accuracy"""

    # from statsmodels.tools.eval_measures import rmse

    # def accuracy(y1,y2):
        
    #     accuracy_df=pd.DataFrame()
        
    #     rms_error = np.round(rmse(y1, y2),1)
        
    #     map_error = np.round(np.mean(np.abs((np.array(y1) - np.array(y2)) / np.array(y1))) * 100,1)
              
    #     accuracy_df=accuracy_df.append({"RMSE":rms_error, "%MAPE": map_error}, ignore_index=True)
        
    #     return accuracy_df

    # Error = accuracy(data3.Close, Predictions[:len(data3.Close)])
    # print("Prediction for Day 1 = {}".format(Predictions[0]))
    # print("\nActual {} for Day 1 = {}".format(feature, data3[feature][0]))
    # print("\nError is :\n",Error)
    # print("\nAccuracy for prediction day 1 is: {} %".format(100 - np.abs((Predictions[0] - data3[feature][0]) / data3[feature][0] ) * 100))

    # Predictions_short = [i[0] for i in Predictions[:len(data3[feature])]]
    # # Predictions_short

    # # data3[feature]

    # accuracy(Predictions_short, data3[feature])

    # for i in range(len(data3[feature])):
    #     print("\nAccuracy for prediction day {} is: {} %".format(i+1, 100 - np.abs((Predictions_short[i] - data3[feature][i]) / data3[feature][i] ) * 100))

    # print(hist.history['loss'])
    Predictions_All[feature] = Predictions

    toc = time.perf_counter()
    print("Time for execution: ", toc - tic)
    

main_toc = time.perf_counter()
print("Total time for all executions: ", main_toc - main_tic)

     |████████████████████████████████| 5.5MB 6.3MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22618 sha256=d3136ef316e8602b675d6626428694729144f8a170f1b1dc14ee7069e4eead5c
  Stored in directory: /root/.cache/pip/wheels/04/98/cc/2702a4242d60bdc14f48b4557c427ded1fe92aedf257d4565c
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
[*********************100%***********************]  1 of 1 completed

In [ ]:
Predictions_All

In [ ]:
with pd.ExcelWriter('APOLLO_12Oct.xlsx') as writer:
    Predictions_All.to_excel(writer, sheet_name=stock)